In [1]:
import torch.optim as optim
import torch.nn as nn
from qiskit.providers.fake_provider import FakeLima
from scripts.data_setup import load_data
from scripts.from_circ_to_numpy import operations_to_features, save_to_json, load_from_json
from scripts.model import create_models, train_and_test_step, save_models, load_models
from qiskit import transpile

In [2]:
n_qubits = 5 # 5 == n_qubits, default value for now because of FakeLima
train_circuits, train_observables, train_ideal_exp_vals, train_noisy_exp_vals, test_circuits, test_observables, test_ideal_exp_vals, test_noisy_exp_vals = load_data('data/circuits/data_small_1')

In [3]:
backend = FakeLima()

In [4]:
X_train, y_train, X_test, y_test = load_from_json('data/features/data_small_1')

In [5]:
# specify these :)
sequence_input_size = 7
sequence_hidden_size = 7
sequence_num_layers = 4
sequence_model_type = 'Transformer'
sequence_dropout = 0.1
ann_hidden_layers = 2
ann_hidden_units = 64
ann_dropout = 0
noisy_first = True

sequence_model, ann = create_models(sequence_input_size, 
                                    sequence_hidden_size, 
                                    sequence_num_layers, 
                                    sequence_model_type, 
                                    sequence_dropout, 
                                    ann_hidden_layers, 
                                    ann_hidden_units, 
                                    ann_dropout, 
                                    noisy_first=noisy_first)

sequence_config = { 
    "input_size": sequence_input_size,
    "hidden_size": sequence_hidden_size,
    "num_layers": sequence_num_layers,
    "model_type": sequence_model_type,
    "dropout": sequence_dropout
}

ann_config = {
    "hidden_layers": ann_hidden_layers,
    "hidden_units": ann_hidden_units,
    "dropout": ann_dropout,
    "noisy_first": noisy_first
}

In [6]:
loss_fn = nn.MSELoss() # leave for now
optimizer = optim.Adam(list(ann.parameters()) + list(sequence_model.parameters()), lr=0.0001)

num_epochs = 5
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

([1.0797961517216472,
  0.40390046776066524,
  0.27408067447555173,
  0.2580500662422044,
  0.24979981884495814],
 [0.7005515098571777,
  0.25552883744239807,
  0.24986431002616882,
  0.23395299911499023,
  0.2347128540277481])

In [7]:
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

([0.24747123757398956,
  0.24334678939166302,
  0.23974091625915736,
  0.2417121731256755,
  0.2367707354194373],
 [0.23534682393074036,
  0.22719328105449677,
  0.22534328699111938,
  0.23125092685222626,
  0.2146570384502411])

In [8]:
train_and_test_step(sequence_model, ann, loss_fn, optimizer, X_train, train_noisy_exp_vals, y_train, X_test, test_noisy_exp_vals, y_test, num_epochs, noisy_first=noisy_first)

([0.23672431824269918,
  0.236687750330561,
  0.23463549744968648,
  0.23091619321445153,
  0.23210708818620715],
 [0.22228321433067322,
  0.21652939915657043,
  0.21182917058467865,
  0.21506023406982422,
  0.21360301971435547])